# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [6]:
import pandas as pd
import requests

# 1) Carregando o arquivo
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contar o número total de registros (linhas)
total_registros = sinasc.shape[0]
print(sinasc.shape)

# Contar o número de registros não duplicados
registros_nao_duplicados = sinasc.drop_duplicates().shape[0]

print(f"O número total de registros na base é: {total_registros}")
print(f"O número de registros não duplicados (únicos) é: {registros_nao_duplicados}, não havendo registros duplicados.")

(27028, 69)
O número total de registros na base é: 27028
O número de registros não duplicados (únicos) é: 27028, não havendo registros duplicados.


In [16]:
# 2) Número de valores missing por variável.

valores_missing = sinasc.isna().sum()
print(valores_missing)

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [24]:
# 3) Seleção da base com as colunas que interessam
df_selecao = sinasc [['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df_selecao

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [78]:
# 3) Refazendo a contagem de valores missings

missing_selecao = df_selecao.isna().sum()
print(missing_selecao)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [55]:
# 4) Removendo todos os registros com Apgar5 não preenchido 

df_limpo = df_selecao.dropna(subset=['APGAR5']).copy()
df_limpo

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [77]:
# 4) 
# Contando o número de linhas e o número de missings

linhas_apgar5 = df_limpo['APGAR5'].shape[0]

missing_final = df_limpo.isna().sum()
print("Nova contagem de valores missing (ausentes) no DataFrame limpo:")
print(missing_final)

Nova contagem de valores missing (ausentes) no DataFrame limpo:


LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
APGAR5_cat    0
dtype: int64

In [57]:
# 5) Substituindo os valores não preenchidos pelo código 9

df_limpo.loc[:, ['ESTCIVMAE', 'CONSULTAS']] = df_limpo[['ESTCIVMAE', 'CONSULTAS']].fillna(9)

# Contagem para verificar o resultado (missings devem ser zero nessas colunas)
print("Contagem de valores missing após a substituição de NaN por 9:")
print(df_limpo[['ESTCIVMAE', 'CONSULTAS']].isnull().sum())

Contagem de valores missing após a substituição de NaN por 9:
ESTCIVMAE    0
CONSULTAS    0
dtype: int64


In [58]:
# 6) Substituindo os valores faltantes da quantitativa (QTDFILVIVO) por zero

df_limpo.loc[:, ['QTDFILVIVO']] = df_limpo[['QTDFILVIVO']].fillna(0)

print("Contagem de valores missing após a substituição de NaN por 0:")
print(df_limpo[['QTDFILVIVO']].isnull().sum())

Contagem de valores missing após a substituição de NaN por 0:
QTDFILVIVO    0
dtype: int64


In [59]:
# 7) Ajustando os dados faltantes nas colunas 'LOCNASC', 'IDADEMAE', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ'

# Avaliando a tabela de classificação Estrutura_SINASC_para_CD, decidiu-se por incluir o valor 9 (ignorado) nos dados ausentes das colunas:
# 'LOCNASC', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ' visto que é uma descrição disponível para esses dados.

df_limpo.loc[:, ['LOCNASC', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ']] = df_limpo[['LOCNASC', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ']].fillna(9)

print("Contagem de valores missing após a substituição de NaN por 9:")
print(df_limpo[['LOCNASC', 'ESCMAE', 'GESTACAO', 'GRAVIDEZ']].isnull().sum())

Contagem de valores missing após a substituição de NaN por 9:
LOCNASC     0
ESCMAE      0
GESTACAO    0
GRAVIDEZ    0
dtype: int64


In [74]:
# Já para a coluna 'IDADEMAE', foi adotado o 'Não preenchido' pelo mesmo motivo.
# Em ambos os casos, não optou-se por usar um dado aproximado pois isso poderia causar uma interpretação errônea das estatísticas em geral.

df_limpo.loc[:, ['IDADEMAE']] = df_limpo[['IDADEMAE']].fillna('Não preenchido')

print("Contagem de valores missing após a substituição de NaN por 'Não preenchido':")
print(df_limpo[['IDADEMAE']].isnull().sum())

Contagem de valores missing após a substituição de NaN por 'Não preenchido':
IDADEMAE    0
dtype: int64


In [72]:
# 8) Classificando o grau de asfixia do bebê

df_limpo.loc[(df_limpo['APGAR5'] >= 8) & (df_limpo['APGAR5'] <= 10),
           'APGAR5_cat'] = 'normal'
df_limpo.loc[(df_limpo['APGAR5'] >= 6) & (df_limpo['APGAR5'] <= 7),
           'APGAR5_cat'] = 'asfixia leve'
df_limpo.loc[(df_limpo['APGAR5'] >= 4) & (df_limpo['APGAR5'] <= 5),
           'APGAR5_cat'] = 'asfixia moderada'
df_limpo.loc[(df_limpo['APGAR5'] >= 0) & (df_limpo['APGAR5'] <= 3),
           'APGAR5_cat'] = 'asfixia severa'

df_limpo[['APGAR5', 'APGAR5_cat']])

,APGAR5,APGAR5_cat
0,10.0,normal
1,9.0,normal
2,10.0,normal
3,10.0,normal
4,10.0,normal
...,...,...
27023,9.0,normal
27024,9.0,normal
27025,10.0,normal
27026,9.0,normal


In [71]:
# 8) Frequências de cada categoria do grau de asfixia do bebê

# Calculando a Frequência Absoluta
frequencia_absoluta = df_limpo['APGAR5_cat'].value_counts().rename('Frequência Absoluta')

# Calculando a Frequência Relativa (em percentual)
frequencia_relativa = (df_limpo['APGAR5_cat'].value_counts(normalize=True) * 100).rename('Frequência Relativa (%)')

# Concatenando as duas Series em um único DataFrame para melhor visualização
frequencia_apgar = pd.concat([frequencia_absoluta, frequencia_relativa], axis=1)

# 4. Arredondar a coluna de percentual e formatar para exibição
frequencia_apgar['Frequência Relativa (%)'] = frequencia_apgar['Frequência Relativa (%)'].round(2).astype(str) + '%'

print("Frequência do APGAR5:")
frequencia_apgar

Frequência do APGAR5:


,Frequência Absoluta,Frequência Relativa (%)
APGAR5_cat,,
normal,26463,98.28%
asfixia leve,320,1.19%
asfixia severa,74,0.27%
asfixia moderada,68,0.25%


In [94]:
# 9) Renomeando as variáveis de interesse para que fiquem no snake case

# Vamos criar um dicionário de mapeamento: 'nome_antigo': 'nome_novo'
mapeamento_corrigido = {
    'locnasc': 'loc_nasc',         
    'idademae': 'idade_mae',       
    'estcivmae': 'est_civ_mae',    
    'escmae': 'esc_mae',           
    'qtdfilvivo': 'qtd_fil_vivo',
    'gestcao' : 'gestacao',
    'gravidez': 'gravidez',
    'consultas': 'consultas',
    'apgar5': 'apgar5',
    'apgar5_cat': 'apgar5_cat'
}

# Agora, vamos renomear as colunas usando o método .rename()
df_limpo.rename(columns=mapeamento_corrigido, inplace=True)

# Conferindo a lista de colunas para confirmar as mudanças
print("Colunas renomeadas no DataFrame df_limpo:")
print(df_limpo.columns.to_list())

Colunas renomeadas no DataFrame df_limpo:
['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_cat']
